In [ ]:
# !pip install plotly panel
# !pip install nbformat 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

pn.extension('plotly') # Load the plotly extension for Panel

In [ ]:
try:
    df_merged = pd.read_csv('transactions_2023_2025_sample.csv')

    # CRUCIAL: Convert 'Date' column to datetime objects
    df_merged['Date'] = pd.to_datetime(df_merged['Date'])

    # Ensure 'Expense/Income' column is clean and consistent (e.g., handle potential variations)
    # This might be needed if your merge process doesn't perfectly standardize this column.
    df_merged['Expense/Income'] = df_merged['Expense/Income'].astype(str).str.strip()

    # Filter expenses and incomes once, after date conversion
    df_transactions_expenses = df_merged[df_merged['Expense/Income'].str.lower() == 'expense'].copy()
    df_transactions_incomes = df_merged[df_merged['Expense/Income'].str.lower() == 'income'].copy()


except FileNotFoundError:
    print(f"Error: The merged CSV file '{CSV_FILE_MERGED}' was not found.")
    print("Please ensure your previous steps (e.g., web interface or merge script) save the merged data to this file.")
    # Create an empty DataFrame to prevent further errors
    df_merged = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])
    df_transactions_expenses = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])
    df_transactions_incomes = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])
except Exception as e:
    print(f"An error occurred while loading or processing the merged data: {e}")
    df_merged = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])
    df_transactions_expenses = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])
    df_transactions_incomes = pd.DataFrame(columns=['Date', 'Name / Description', 'Expense/Income', 'Amount (EUR)', 'Category', 'Transaction vs category'])

In [ ]:
# Get all unique years present in the DataFrame
# This will work even if df_merged is empty, resulting in an empty list.
if not df_merged.empty:
    unique_years = sorted(df_merged['Date'].dt.year.unique())
else:
    unique_years = [] # No years if DataFrame is empty

In [ ]:
# Function to generate all expense bar plots per year ---

def get_all_expenses_bar_plots():
    # Bar Plot
    plots_column = []
    if df_transactions_expenses.empty:
        plots_column.append(pn.pane.Markdown("### No expense data available."))
        return pn.Column(*plots_column)

    for year in unique_years:
        df_filtered_expenses_year = df_transactions_expenses[df_transactions_expenses['Date'].dt.year == year].copy()

        if not df_filtered_expenses_year.empty:
            # Group by month and sum amounts for the current year
            df_plot_expenses = df_filtered_expenses_year.groupby(df_filtered_expenses_year['Date'].dt.month_name())['Amount (EUR)'].sum().reset_index()
            df_plot_expenses.columns = ['Month', 'Amount']

            # Order by month for chronological display
            month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
            df_plot_expenses['Month'] = pd.Categorical(df_plot_expenses['Month'], categories=month_order, ordered=True)
            df_plot_expenses = df_plot_expenses.sort_values('Month')

            fig = px.bar(df_plot_expenses, x='Month', y='Amount', title=f'Expenses for {year}',
                         color_discrete_sequence=['#FF6347']) # Ejemplo de color rojo tomate para gastos)
            plots_column.append(fig)
        else:
            plots_column.append(pn.pane.Markdown(f"### No expense data available for {year}"))

    return pn.Column(*plots_column) # Returns a column of all plots

In [ ]:
# Function to generate all income bar plots per year ---

def get_all_incomes_bar_plots():
    plots_column = []
    if df_transactions_incomes.empty:
        plots_column.append(pn.pane.Markdown("### No income data available."))
        return pn.Column(*plots_column)

    for year in unique_years:
        df_filtered_incomes_year = df_transactions_incomes[df_transactions_incomes['Date'].dt.year == year].copy()

        if not df_filtered_incomes_year.empty:
            # Group by month and sum amounts for the current year
            df_plot_incomes = df_filtered_incomes_year.groupby(df_filtered_incomes_year['Date'].dt.month_name())['Amount (EUR)'].sum().reset_index()
            df_plot_incomes.columns = ['Month', 'Amount']

            # Order by month
            month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
            df_plot_incomes['Month'] = pd.Categorical(df_plot_incomes['Month'], categories=month_order, ordered=True)
            df_plot_incomes = df_plot_incomes.sort_values('Month')

            fig = px.bar(df_plot_incomes, x='Month', y='Amount', title=f'Incomes for {year}',
                         color_discrete_sequence=['#32CD32']) # Ejemplo de color verde lima para ingresos)
            plots_column.append(fig)
        else:
            plots_column.append(pn.pane.Markdown(f"### No income data available for {year}"))

    return pn.Column(*plots_column) # Returns a column of all plots

In [ ]:
def get_all_expense_pie_charts_by_category():
    plots_pie_column = []
    if df_transactions_expenses.empty:
        plots_pie_column.append(pn.pane.Markdown("### No expense category data available."))
        return plots_pie_column # Return list for consistency

    for year in unique_years:
        df_filtered_expenses_year = df_transactions_expenses[df_transactions_expenses['Date'].dt.year == year].copy()

        if not df_filtered_expenses_year.empty and 'Category' in df_filtered_expenses_year.columns:
            df_plot_pie = df_filtered_expenses_year.groupby('Category')['Amount (EUR)'].sum().reset_index()
            df_plot_pie.columns = ['Category', 'Total Amount']

            if not df_plot_pie.empty:
                df_plot_pie.columns = ['Category', 'Total Amount']
                
                # Calculate the total sum for the center
                total_sum = df_plot_pie['Total Amount'].sum()

            fig = px.pie(df_plot_pie, values='Total Amount', names='Category',
                         title=f'Expense per Category for {year}',
                         hole=0.3,
                         color_discrete_sequence=px.colors.qualitative.Pastel) # Usa una paleta pastel de Plotly)
            
            # ADDING TEXT TO THE CENTER OF THE DONUT CHART 
            fig.update_traces(textposition='inside', direction = 'clockwise', hole=0.3, textinfo="label+percent")

            fig.update_layout(uniformtext_minsize=10,
                          # Add annotations in the center of the donut.
                          annotations=[
                              dict(
                                  text=f'Total:<br>{total_sum:.2f}€',
                                  # Square unit grid starting at bottom left of page
                                  x=0.5, y=0.5, font_size=14,
                                  # Hide the arrow that points to the [x,y] coordinate
                                  showarrow=False
                              )
                          ]
                          )
            plots_pie_column.append(fig)
        else:
            plots_pie_column.append(pn.pane.Markdown(f"### No expense category data available for {year} or 'Category' column missing."))
            
    return plots_pie_column # Return a list of all pie charts

In [ ]:
def get_all_income_pie_charts_by_category():
    plots_pie_column = []
    if df_transactions_incomes.empty:
        plots_pie_column.append(pn.pane.Markdown("### No expense category data available."))
        return plots_pie_column # Return list for consistency

    for year in unique_years:
        df_filtered_incomes_year = df_transactions_incomes[df_transactions_incomes['Date'].dt.year == year].copy()

        if not df_filtered_incomes_year.empty and 'Category' in df_filtered_incomes_year.columns:
            df_plot_pie = df_filtered_incomes_year.groupby('Category')['Amount (EUR)'].sum().reset_index()
            df_plot_pie.columns = ['Category', 'Total Amount']

            if not df_plot_pie.empty:
                df_plot_pie.columns = ['Category', 'Total Amount']
                
                # Calculate the total sum for the center
                total_sum = df_plot_pie['Total Amount'].sum()

            fig = px.pie(df_plot_pie, values='Total Amount', names='Category',
                         title=f'Income per Category for {year}',
                         hole=0.3,
                         color_discrete_sequence=px.colors.qualitative.Pastel) # Usa una paleta pastel de Plotly)
            
            # ADDING TEXT TO THE CENTER OF THE DONUT CHART 
            fig.update_traces(textposition='inside', direction = 'clockwise', hole=0.3, textinfo="label+percent")

            fig.update_layout(uniformtext_minsize=10,
                          # Add annotations in the center of the donut.
                          annotations=[
                              dict(
                                  text=f'Total:<br>{total_sum:.2f}€',
                                  # Square unit grid starting at bottom left of page
                                  x=0.5, y=0.5, font_size=14,
                                  # Hide the arrow that points to the [x,y] coordinate
                                  showarrow=False
                              )
                          ]
                          )
            plots_pie_column.append(fig)
        else:
            plots_pie_column.append(pn.pane.Markdown(f"### No expense category data available for {year} or 'Category' column missing."))
            
    return plots_pie_column # Return a list of all pie charts

In [ ]:
# Combine bar plots and the new pie chart for the Expenses tab
def get_expenses_tab_content():
    # Get all yearly bar charts
    bar_charts = get_all_expenses_bar_plots()
    # Get all yearly pie charts
    pie_charts = get_all_expense_pie_charts_by_category()
    
    # Return a column containing all these plots (bar charts first, then pie charts)
    return pn.Column(
        pn.pane.Markdown("## Monthly Expenses Overview"), # Optional title for this section
        *bar_charts, # Unpack the list of bar charts
        pn.pane.Markdown("---"), # Optional separator
        pn.pane.Markdown("## Expense Distribution by Category"), # Optional title for this section
        *pie_charts # Unpack the list of pie charts
    )

In [ ]:
# Combine bar plots and the new pie chart for the Incomes tab
def get_incomes_tab_content():
    # Get all yearly bar charts
    bar_charts = get_all_incomes_bar_plots()
    # Get all yearly pie charts
    pie_charts = get_all_income_pie_charts_by_category()

    return pn.Column(
    pn.pane.Markdown("## Monthly Incomes Overview"), # Optional title for this section
    *bar_charts, # Unpack the list of bar charts
    pn.pane.Markdown("## Income Distribution by Category"),  # Optional title for this section
    *pie_charts # Unpack the list of pie charts
    )

In [ ]:
# Create the Panel Dashboard ---

tabs = pn.Tabs(
    ('Expenses', get_expenses_tab_content),
    ('Incomes', get_incomes_tab_content)
)

template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"),
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                )
                ],
    header_background="c0b9dd",
)

template.show()